<img src="https://heig-vd.ch/docs/default-source/doc-global-newsletter/2020-slim.svg" alt="Logo HEIG-VD" style="width: 80px;" align="right"/>

# Cours APN - Labo 5 : Cartes auto-organisatrices (SOM) de Kohonen

## Résumé

Le but de ce laboratoire est de réaliser une expérience de groupement avec des cartes auto-organisatrices.  Le code vous est fourni, y compris les deux librairies `kohonen.py` et `KohonenUtils.py`.  On vous demande de trouver une bonne paramétrisation de la carte, et d'expliquer brièvement quelques éléments de code.

In [ ]:
import KohonenUtils
import numpy as np
import pandas as pd
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
%matplotlib inline

## 1. Données

On considère une liste de 16 animaux, décrits à l'aide de 13 attributs booléens.  Les données sont fournies dans cette section.  Veuillez exécuter les cellules, inspecter les données, et répondre à la question finale.

In [ ]:
matrix = np.array([[1,0,0,1,0,0,0,0,1,0,0,1,0],
                   [1,0,0,1,0,0,0,0,1,0,0,0,0],
                   [1,0,0,1,0,0,0,0,1,0,0,0,1],
                   [1,0,0,1,0,0,0,1,1,0,0,1,1],
                   [1,0,0,1,0,0,0,0,1,1,0,1,0],
                   [1,0,0,1,0,0,0,0,1,1,0,1,0],
                   [0,1,0,1,0,0,0,0,1,1,0,1,0],
                   [0,1,0,0,1,1,0,0,0,1,0,0,0],
                   [0,1,0,0,1,1,0,0,0,0,1,0,0],
                   [0,1,0,0,1,1,0,1,0,1,1,0,0],
                   [1,0,0,0,1,1,0,0,0,1,0,0,0],
                   [0,0,1,0,1,1,0,0,0,1,1,0,0],
                   [0,0,1,0,1,1,0,1,0,1,1,0,0],
                   [0,0,1,0,1,1,1,1,0,0,1,0,0],
                   [0,0,1,0,1,1,1,1,0,0,1,0,0],
                   [0,0,1,0,1,1,1,0,0,0,0,0,0]])

In [ ]:
animals = pd.DataFrame(matrix.astype(float),columns=['small', 'medium', 'big', '2legs', '4legs', 'hair', 'hooves',
                                'mane', 'feathers', 'hunt', 'run', 'fly', 'swim'])

animals.index = ['dove', 'hen', 'duck', 'goose', 'owl', 'hawk', 'eagle', 'fox',
                 'dog', 'wolf', 'cat', 'tiger', 'lion', 'horse', 'zebra', 'cow']

display(animals)

In [ ]:
plt.figure(figsize=(3,3))
plt.xticks(np.arange(matrix.shape[1]), animals.columns, rotation=90, fontsize=8)
plt.yticks(np.arange(matrix.shape[0]), animals.index, fontsize=8)
plt.imshow(matrix, cmap='binary')

**Question 1a :** pouvez-vous identifier visuellement des groupes d'animaux ?

In [ ]:
# Votre réponse ici :


## 2. Entraînement d'une carte de Kohonen

Le code fourni ci-dessous crée une carte auto-organisatrice, l'entraîne, et affiche le résultat final sous la forme d'une matrice-U.  Veuillez répondre aux questions qui suivent.

In [ ]:
kmap = KohonenUtils.KohonenMap(5, 7, size_vector=len(animals.columns)) # initialiser la SOM

n_iter = 10

learning_rate = KohonenUtils.ExponentialTimeseries(1, 0.2, n_iter * len(animals.index))

neighborhood_size = KohonenUtils.ExponentialTimeseries(4./3 * len(animals.index), 1, n_iter * len(animals.index))

kmap.train(animals.values, np.array(animals.index), n_iter, learning_rate, neighborhood_size) # entraîner la SOM

kmap.plot_umatrix(plot_names=True, plot_empty=True, fontsize=10) # afficher le résultat

**Question 2a :** veuillez expliquer brièvement, avec vos propres mots, ce que représente la matrice U, en répondant aux questions suivantes :
- que représentent les trois paramètres du constructeur `KohonenMap` ?
- que représentent les points bleus ?
- que représentent les points rouges ?
- que représentent les points gris et les niveaux de gris ?
- pour une carte de taille _m_ * _n_, combien y a-t-il de points gris ?

In [ ]:
# Votre réponse ici :


**Question 2b :** veuillez régler les paramètres du constructeur `KohonenMap` et le `n_iter` de façon à aboutir à une carte de Kohonen qui regroupe les seize animaux d'une manière qui vous semble cohérente.   Veuillez indiquer ci-dessous les paramètres choisis, lister les groupes observés d'animaux, et commenter le résultat.

In [ ]:
# Votre réponse ici :


**Question 2c :** veuillez afficher une image de la matrice-U, au format PNG ou JPG, sur laquelle vous aurez dessiné les frontières visibles (à la main, avec un logiciel d'édition de votre choix).  Veillez à sauvegarder le notebook avec l'image avant l'envoi sur Cyberlearn.

In [ ]:
# Pour afficher votre image ici :
plt.imshow(mpimg.imread('matrice-u.png'))

## 3. Analyse des paramètres de l'entraînement

Dans cette section, le but est de comprendre les paramètres `learning_rate` et `neighborhood_size`.  Veuillez ré-exécuter les commandes ci-dessous qui les ré-initialisent.  On vous encourage à faire différents essais avant de répondre.

In [ ]:
learning_rate = KohonenUtils.ExponentialTimeseries(1, 0.2, n_iter * len(animals.index))

**Question 3a :** veuillez expliquer avec vos propres mots le fonctionnement de `learning_rate` en répondant aux questions suivantes :
- quel est le type de `learning_rate` ?
- quelles sont les significations de ses trois paramètres ?
- pourquoi la valeur du troisième paramètre est-elle choisie ainsi ?
- comme fonctionne la `learning_rate` ?

In [ ]:
# Votre réponse ici :


**Question 3b :** veuillez afficher d'une façon convenable la ou les valeur(s) de la `learning_rate`.

In [ ]:
# Votre code pour l'affichage de learning_rate ici :


On pose maintenant les mêmes questions pour `neighborhood_size`.  <br/> Si les réponses sont similaires, vous pouvez copier/coller les réponses précédentes.

In [ ]:
neighborhood_size = KohonenUtils.ExponentialTimeseries(4./3 * len(animals.index), 1, n_iter * len(animals.index))

**Question 3c :** veuillez expliquer avec vos propres mots le fonctionnement de `neighborhood_size` en répondant aux questions suivantes :
- quel est le type de `neighborhood_size` ?
- quelles sont les significations de ses trois paramètres ?
- comme fonctionne la `neighborhood_size` ?

In [ ]:
# Votre réponse ici :


**Question 3d :** veuillez afficher d'une façon convenable la ou les valeur(s) de la `neighborhood_size`.

In [ ]:
# Votre code pour l'affichage de neighborhood_size ici :


**Fin du Labo 5.**  Veuillez nettoyer ce notebook, afficher les résultats et les commentaires demandés, l'enregistrer, et le soumettre comme devoir sur Cyberlearn.